<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/KERAS_MODELS_BUILDING_BLOCKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [20]:
#Building keras layers from scratch
class MyLayer(keras.layers.Layer):
  def __init__(self, units = 64, input_shape=64):
    super(MyLayer, self).__init__()
    w_initial = tf.random_normal_initializer()
    b_initial = tf.zeros_initializer()
    self.w = tf.Variable(initial_value = w_initial(shape = (units, input_shape)), dtype = 'float32', trainable = True)
    self.b = tf.Variable(initial_value = b_initial(shape = (units, )), dtype = 'float32', trainable = True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

x = tf.ones(shape = (3,3))
layer1 = MyLayer(3,1)
out = layer1(x)
print(out)
assert layer1.weights == [layer1.w, layer1.b]

tf.Tensor(
[[-0.01079231 -0.01079231 -0.01079231]
 [-0.01079231 -0.01079231 -0.01079231]
 [-0.01079231 -0.01079231 -0.01079231]], shape=(3, 3), dtype=float32)


In [21]:
#Altenative way to add parameters in a keras layer
class AltLayer(keras.layers.Layer):
  def __init__(self, input_shape = 64, units = 64):
    super(AltLayer, self).__init__()
    self.w = self.add_weight(shape = (input_shape, units), initializer = 'random_normal',trainable = True)
    self.b = self.add_weight(shape = (units,), initializer = 'zeros', trainable = True)
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

x = tf.ones(shape = (3,3))
layer2 = AltLayer(3,1)
out = layer2(x)
print(out)
assert layer2.weights == [layer2.w, layer2.b]

tf.Tensor(
[[-0.09851134]
 [-0.09851134]
 [-0.09851134]], shape=(3, 1), dtype=float32)


In [33]:
#Adding the input shape in a separate block
class AddInput(keras.layers.Layer):
  def __init__(self, units):
    super(AddInput, self).__init__()
    self.units = units
  
  def build(self, input_shape):
    self.w = self.add_weight(
        shape = (input_shape[-1], self.units),
        initializer = 'random_normal',
        trainable = True
    )
    self.b = self.add_weight(
        shape = (self.units,),
        initializer = 'zeros',
        trainable = True
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

layer3 = AddInput(32)
out = layer3(x)
print(out)
assert layer3.weights == [layer3.w, layer3.b]

tf.Tensor(
[[-0.0553377   0.04070884 -0.0115709  -0.00817596 -0.02025527 -0.01317053
  -0.00115759  0.06118619  0.01554302  0.1798982   0.09257257  0.0174687
  -0.04247794  0.03030066 -0.02307886 -0.00474579  0.02217307  0.00970889
   0.15911117 -0.03221453 -0.02186662  0.19686139  0.01931957  0.01819673
  -0.01008678 -0.0936542  -0.1558387  -0.03757554  0.11727723 -0.04991619
   0.07638891 -0.02369768]
 [-0.0553377   0.04070884 -0.0115709  -0.00817596 -0.02025527 -0.01317053
  -0.00115759  0.06118619  0.01554302  0.1798982   0.09257257  0.0174687
  -0.04247794  0.03030066 -0.02307886 -0.00474579  0.02217307  0.00970889
   0.15911117 -0.03221453 -0.02186662  0.19686139  0.01931957  0.01819673
  -0.01008678 -0.0936542  -0.1558387  -0.03757554  0.11727723 -0.04991619
   0.07638891 -0.02369768]
 [-0.0553377   0.04070884 -0.0115709  -0.00817596 -0.02025527 -0.01317053
  -0.00115759  0.06118619  0.01554302  0.1798982   0.09257257  0.0174687
  -0.04247794  0.03030066 -0.02307886 -0.00474579 

In [37]:
#Stacking keras layers together
class StackLayer(keras.layers.Layer):
  def __init__(self):
    super(StackLayer, self).__init__()
    self.ln1 = AddInput(32)
    self.ln2 = AddInput(16)
    self.ln3 = AddInput(1)

  def call(self,inputs):
    x = self.ln1(inputs)
    x = tf.nn.relu(x)
    x = self.ln2(x)
    x = tf.nn.relu(x)
    return self.ln3(x)
  
model= StackLayer()
model(tf.ones(shape = (3,32)))
print(f"trainable pars: {len(model.weights)}\nNon-trainable pars: {len(model.trainable_weights)}")

    


trainable pars: 6
Non-trainable pars: 6


In [51]:
#Add Loss as a regularizer to the keras layer( This can be added as additional loss to the main loss)
class AddLoss(keras.layers.Layer):
  def __init__(self,rate = 1e-3):
    super(AddLoss, self).__init__()
    self.rate = rate
  
  def call(self, inputs):
    self.add_loss(self.rate * tf.reduce_sum(inputs))
    return inputs



In [54]:
#retrieve the loss at any stage using layer.losses
class LossRetrieve(keras.layers.Layer):
  def __init__(self):
    super(LossRetrieve, self).__init__()
    self.myregulator = AddLoss(1e-3)
  
  def call(self, inputs):
    return self.myregulator(inputs)


los1 = LossRetrieve()
_=los1(tf.zeros((1,1)))
assert len(los1.losses) == 1

In [56]:
#Adding kernel regularizer to the layer
class WithKernel(keras.layers.Layer):
  def __init__(self):
    super(WithKernel, self).__init__()
    self.mydense = keras.layers.Dense(units = 32,
                                      kernel_regularizer = tf.keras.regularizers.l2(1e-3))
  
  def call(self, inputs):
    return self.mydense(inputs)

ken = WithKernel()
_= ken(tf.zeros((1,1)))
print(ken.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0013316458>]


In [59]:
#Using the above defined losses (loss, regularizer to train keras model)
inputs = keras.Input(shape = (3,))
outputs = AddLoss()(inputs)
model = keras.Model(inputs = inputs, outputs = outputs)
model.compile(loss = 'mse', optimizer = 'adam')#with the main loss also user defined losses are added
model.summary()
model.fit(np.random.random((64,3)), np.random.random((64,3)))


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
add_loss_3 (AddLoss)         (None, 3)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
2/2 [==============================] - 0s 1ms/step - loss: 0.2341


In [61]:
#Training using only the user defined loss
inputs = keras.Input(shape = (32,))
outputs = AddLoss()(inputs)
model1 = keras.Model(inputs = inputs, outputs = outputs)
model1.compile(optimizer = 'adam')
model1.fit(np.random.random(size = (64,32)), np.random.random(size = (64,32)))

2/2 [==============================] - 0s 2ms/step - loss: 0.5045


In [82]:
#Adding a metric to the keras model to track moving average of a quantity during training
#This compute the accuracy of the model
class LogisticLoss(keras.layers.Layer):
  def __init__(self, name = None):
    super(LogisticLoss, self).__init__(name = name)
    self.logloss = keras.losses.BinaryCrossentropy(from_logits=True)
    self.accuracy1 = keras.metrics.BinaryAccuracy()
  
  def call(self, y_hat, y_orig, sample_inputs = None):
    loss1 = self.logloss(y_hat, y_orig, sample_inputs)
    self.add_loss(loss1)
    accuracy2 = self.accuracy1(y_hat, y_orig, sample_inputs)
    self.add_metric(accuracy2, name = 'accuracy')
    return tf.nn.softmax(logits=y_hat)




In [83]:
mymetrics = LogisticLoss()
y_hat = tf.ones(shape = (3,3))
y_orig = tf.ones(shape = (3,3))
out_loss = mymetrics(y_hat, y_orig)
print(f"Layer_metric: {mymetrics.metrics}\ncurrent accuracy: {float(mymetrics.metrics[0].result())}")


Layer_metric: [<tensorflow.python.keras.metrics.BinaryAccuracy object at 0x7fb74fc692e8>]
current accuracy: 1.0
